In [3]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import requests
import zipfile
import os
import random

In [8]:
#function to download the DEM
def download_dem(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
    with zipfile.ZipFile(save_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.join(out_dir, 'dem'))

In [13]:
#set URL and directory, run download function
dem_url = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'
out_dir = r"C:\Users\SCHIL726\Downloads"

download_dem(dem_url, os.path.join(out_dir, 'dem.zip'))

In [1]:
#clip data to hennepin county because there is a lot
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb"):
    out_raster = arcpy.sa.ExtractByMask(
        in_raster="digital_elevation_model_30m",
        in_mask_data="hennepin_polygon",
        extraction_area="INSIDE",
        analysis_extent='439324.861 4959173.419 486049.45 5010478.118 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
    )
    out_raster.save(r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb\hennepin_dem_30m")

In [2]:
#convert raster to points
arcpy.conversion.RasterToPoint(
    in_raster="hennepin_dem_30m",
    out_point_features=r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb\MN_dem_30m_points",
    raster_field="VALUE"
)

<Result 'C:\\Users\\SCHIL726\\Documents\\ArcGIS\\Projects\\GIS5572_Lab3\\GIS5572_Lab3.gdb\\MN_dem_30m_points'>

In [5]:
#sample 10% of the points - justifying this amount because there are a LOT (still almost 200k just in hennepin)

#find the point count first
point_count = int(arcpy.GetCount_management("MN_dem_30m_points").getOutput(0))

sample_calc = int(point_count * 0.1)

#create index of points them randomly sample it
index = list(range(1,point_count + 1))
sampled_index = random.sample(index, sample_calc)

print(len(sampled_index))

174500


In [6]:
#create feature class to store sampled points, using hennepin_dem_30m as template
arcpy.management.CreateFeatureclass(
    out_path=r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb",
    out_name="dem_sampled",
    geometry_type="POINT",
    template="MN_dem_30m_points",
    has_m="DISABLED",
    has_z="DISABLED",
    spatial_reference='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision',
    config_keyword="",
    spatial_grid_1=0,
    spatial_grid_2=0,
    spatial_grid_3=0,
    out_alias="",
    oid_type="SAME_AS_TEMPLATE"
)

<Result 'C:\\Users\\SCHIL726\\Documents\\ArcGIS\\Projects\\GIS5572_Lab3\\GIS5572_Lab3.gdb\\dem_sampled'>

In [7]:
#store sample points in an array - will then be moved to the fc just created
points = []
with arcpy.da.SearchCursor("MN_dem_30m_points", ["SHAPE@XY", "grid_code"]) as cursor:
    for row in cursor:
        points.append(row)

#randomly selected
random.shuffle(points)

sampled_points = points[:sample_calc]

In [9]:
#add sampled points to dem_sampled fc
with arcpy.da.InsertCursor("dem_sampled", ["SHAPE@XY", "pointid", "grid_code"]) as cursor:
    for idx, point in enumerate(sampled_points):
        x, y = point[0]
        point_geometry = arcpy.PointGeometry(arcpy.Point(x, y))
        cursor.insertRow([point_geometry, idx+1, point[1]])

In [10]:
#interpolation #1 - IDW
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb"):
    out_raster = arcpy.sa.Idw(
        in_point_features="dem_sampled",
        z_field="grid_code",
        cell_size=186.24,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb\IDW_sampled_dem")

In [11]:
#interpolation #2 - ordinary kriging
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="dem_sampled",
        z_field="grid_code",
        kriging_model="Spherical # # # #",
        cell_size=186.24,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb\ordinary_kriging_sampled_dem")

In [12]:
#interpolation 3 - universal kriging
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="dem_sampled",
        z_field="grid_code",
        kriging_model="LinearDrift 186.240000 # # #",
        cell_size=186.24,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb\universal_kriging_sampled_dem")

In [13]:
#run exploratory interpolation for statistical analysis
arcpy.ga.ExploratoryInterpolation(
    in_features="dem_sampled",
    value_field="grid_code",
    out_cv_table=r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb\ExploratoryInterpolation",
    out_geostat_layer="highest_rank_dem",
    interp_methods="ORDINARY_KRIGING;UNIVERSAL_KRIGING;IDW",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result 'C:\\Users\\SCHIL726\\Documents\\ArcGIS\\Projects\\GIS5572_Lab3\\GIS5572_Lab3.gdb\\ExploratoryInterpolation'>

In [1]:
#universal kriging shows as best per exploratory interpolation table
#but based on the SA interpolation rasters in the GUI ordinary kriging looks best

#caculate original dem and ordinary kriging dem difference with raster math
difference_raster = arcpy.sa.Minus("hennepin_dem_30m", "ordinary_kriging_sampled")

In [19]:
#convert difference raster to points
arcpy.conversion.RasterToPoint(
    in_raster="difference_raster",
    out_point_features=r"C:\Users\SCHIL726\Documents\ArcGIS\Projects\GIS5572_Lab3\GIS5572_Lab3.gdb\difference_dem_points",
    raster_field="Value"
)

<Result 'C:\\Users\\SCHIL726\\Documents\\ArcGIS\\Projects\\GIS5572_Lab3\\GIS5572_Lab3.gdb\\difference_dem_points'>

In [23]:
#rename "grid_code" field to "elevation_difference"
field_mapping = arcpy.FieldMappings()
field_mapping.addTable("difference_dem_points")
for field in field_mapping.fields:
     if field.name == "grid_code":
        field.name = "elevation_difference"
        arcpy.management.AlterField("difference_dem_points", "grid_code", "elevation_difference")

In [4]:
#push elevation interpolation(ordinary_kriging_sampled_dem_points) to database
fc1 = "ordinary_kriging_sampled_dem_points"

#connect to db
conn = psycopg2.connect(
    dbname="gis_data",
    user="postgres",
    password=" ",
    host="34.30.71.239",
    port="5432"
)
cur = conn.cursor()

#recreate table (since i can't push a gdf to postgis in arcpy notebook)
cur.execute("DROP TABLE IF EXISTS ordinary_kriging_sampled_dem_points;")
cur.execute("""
    CREATE TABLE ordinary_kriging_sampled_dem_points (
        id SERIAL PRIMARY KEY,
        pointid INTEGER,
        grid_code DOUBLE PRECISION,
        geom GEOMETRY(POINT, 4326)
    );
""")
conn.commit()

#read from the feature class with arcpy
with arcpy.da.SearchCursor(fc1, ["SHAPE@XY", "pointid", "grid_code"]) as cursor:
    for row in cursor:
        (x, y) = row[0]
        pointid = row[1]
        grid_code = row[2]
        cur.execute("""
            INSERT INTO ordinary_kriging_sampled_dem_points (pointid, grid_code, geom)
            VALUES (%s, %s, ST_SetSRID(ST_Point(%s, %s), 4326));
        """, (pointid, grid_code, x, y))

conn.commit()
cur.close()
conn.close()

In [5]:
#push elevation point accuracy assessment(difference_dem_points) to database
fc2 = "difference_dem_points"

#reconnect to db
conn = psycopg2.connect(
    dbname="gis_data",
    user="postgres",
    password=" ",
    host="34.30.71.239",
    port="5432"
)
cur = conn.cursor()

#recreate table (since i can't push a gdf to postgis in arcpy notebook)
cur.execute("DROP TABLE IF EXISTS difference_dem_points;")
cur.execute("""
    CREATE TABLE difference_dem_points (
        id SERIAL PRIMARY KEY,
        pointid INTEGER,
        elevation_difference DOUBLE PRECISION,
        geom GEOMETRY(POINT, 4326)
    );
""")
conn.commit()

#read from the feature class with arcpy
with arcpy.da.SearchCursor(fc2, ["SHAPE@XY", "pointid", "elevation_difference"]) as cursor:
    for row in cursor:
        (x, y) = row[0]
        pointid = row[1]
        diff = row[2]
        cur.execute("""
            INSERT INTO difference_dem_points (pointid, elevation_difference, geom)
            VALUES (%s, %s, ST_SetSRID(ST_Point(%s, %s), 4326));
        """, (pointid, diff, x, y))

conn.commit()
cur.close()
conn.close()